In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from core.s3_utils import S3_Bucket
import pandas as pd
import re
import plotly.express as px

In [ ]:
list_files = S3_Bucket().list_responses_keys_of_brand('stellantis')

In [ ]:
df_stellantis = pd.DataFrame()
for index, file in list_files.iterrows():
   res = re.findall("[A-Z]\w+", file['key'])
   if len(res) > 0:
      if res[0] in ['VR3UKZKWZPJ911695','VR3UKZKWZPJ918303','VR3UKZKWZPJ842705', 'VR3UKZKWZPJ911696','VR3UKZKWZPJ918307','VR3UKZKWZPJ789946',
                    'VR3UKZKW6RJ619807','VR3UKZKWZPJ911694', 'VR3UKZKWZPJ853581', 'VR3UKZKWZPJ789947', 'VR3UKZKW5RJ625386', 'VR3UKZKWZPJ911720']:
         data = S3_Bucket().read_json_file(file['key'])
         vin = data.get('vin')
         odometer = data.get('odometer', '')
         soh = data.get("electricity", {}).get('battery', {}).get('stateOfHealth', '')

         if (len(odometer) > 0) &  (len(soh) > 0):
            odometer = pd.DataFrame(odometer).drop_duplicates()
         
            soh = pd.DataFrame(soh).drop_duplicates()
         
            odometer['datetime'] = pd.to_datetime(odometer['datetime'])
            soh['datetime'] = pd.to_datetime(soh['datetime'])
   
            df = pd.merge_asof(odometer.sort_values('datetime'),soh.sort_values('datetime'), on='datetime', direction='nearest')
            df['vin'] = vin
            df_stellantis = pd.concat((df_stellantis, df))
            
         
      

In [ ]:
df_stellantis

In [ ]:
import plotly.express as px

In [ ]:
px.scatter(df_stellantis, x='value', y='percentage', color='vin')